# 02 – Forecasting Cash Flow and CVaR Buffer Sizing

Objective:
- Forecast net operating cash flows.
- Compute CVaR-based buffer to protect against shortfall risk.

In [2]:
# Easy fix for finding parent directory modules
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from value_ops_lab.synth import make_synthetic
from value_ops_lab.risk_models import quantile_forecast, cvar_cash_buffer

# Generate synthetic dataset
df = make_synthetic(n_months=36)
df["net_cf"] = df["sales"] - df["cogs"]

# Features: simple lag features
df["lag1"] = df["net_cf"].shift(1)
df["lag2"] = df["net_cf"].shift(2)
df = df.dropna()

X = df[["lag1","lag2"]]
y = df["net_cf"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Forecast next 6 months
X_future = X_test.tail(6)
preds = quantile_forecast(X_train, y_train, X_future, quantiles=(0.1,0.5,0.9))
preds

{0.1: array([16585.44902952, 19409.66625873, 19409.66625873, 17359.46424146,
        17359.46424146, 12807.48864039]),
 0.5: array([25285.5813004 , 19599.28015337, 23335.13618029, 21738.57894345,
        29620.450586  , 23926.62398049]),
 0.9: array([26837.19650362, 23573.34675724, 23573.34675724, 29228.14761738,
        31499.0353344 , 24194.00799946])}

In [ ]:
# CVaR buffer sizing (robust to any test size)
n = min(12, len(y_test))                 # use up to 12, but not more than available
noise = np.random.normal(0, 5000, n)     # same length as y_test tail
scenarios = y_test.tail(n).to_numpy() + noise

from value_ops_lab.risk_models import cvar_cash_buffer

try:
    buffer, t = cvar_cash_buffer(scenarios, alpha=0.95)
    print(f"Optimal buffer at 95% CVaR: ${buffer:,.0f}")
except ImportError:
    print("cvxpy not installed — skipping CVaR optimization.")

ValueError: operands could not be broadcast together with shapes (7,) (12,) 

### Key Insight
- The CVaR buffer balances cost of capital vs. probability of cash shortfall.
- This aligns with **risk-aware FP&A**, highly relevant to Accordion’s analytics.